# Chapter 52: Model Serving Infrastructure
Request batching and auto-scaling.


In [ ]:
import random
print("Model Serving ready")


In [ ]:
class Batcher:
    def __init__(self,max_b=16): self.q=[]; self.max_b=max_b
    def add(self,rid,tok): self.q.append({"id":rid,"tok":tok})
    def get(self): batch=self.q[:self.max_b]; self.q=self.q[self.max_b:]; return batch
    def infer(self,batch):
        if not batch: return []
        total_tok=sum(r["tok"] for r in batch)
        lat=10+total_tok*0.05+random.gauss(0,2)
        return [{"id":r["id"],"lat_ms":round(lat,1)} for r in batch]
random.seed(42); b=Batcher(8)
for i in range(32): b.add("req-"+str(i),random.randint(100,500))
all_r=[]
while b.q: all_r.extend(b.infer(b.get()))
print("Processed",len(all_r),"requests")
avg_lat=sum(r["lat_ms"] for r in all_r)/len(all_r)
print("Avg latency:",round(avg_lat,1),"ms")


In [ ]:
print("Batch | Latency(ms) | Req/s | GPU%")
print("-"*40)
for bs in [1,2,4,8,16,32]:
    lat=15.0+bs*0.8; thr=bs/(lat/1000); gpu=min(0.95,bs/32)
    print(str(bs)+" | "+str(round(lat,1))+" | "+str(round(thr,1))+" | "+str(round(gpu*100,0))+"%")
class AutoScaler:
    def __init__(self,mn=1,mx=10,tgt=100): self.r=mn; self.mn=mn; self.mx=mx; self.t=tgt
    def scale(self,qps):
        d=max(self.mn,min(self.mx,round(qps/self.t+0.5)))
        a="up" if d>self.r else "down" if d<self.r else "none"
        self.r=d; return {"replicas":self.r,"action":a}
as_=AutoScaler(2,8,50)
for qps in [20,60,120,200,80,30]:
    r=as_.scale(qps); print("QPS="+str(qps)+" -> replicas="+str(r["replicas"])+" ("+r["action"]+")")
